In [27]:

from __future__ import print_function
import torchvision.datasets.vision as vision
import warnings
from PIL import Image
import os
import os.path
import random
import numpy as np
import torch
import codecs
from torchvision.datasets.utils import download_url, download_and_extract_archive, extract_archive, \
    makedir_exist_ok, verify_str_arg

class MyMNIST(vision.VisionDataset):

    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False, limit_data=None, resize=False):
        super(MyMNIST, self).__init__(root, transform=transform,
                                    target_transform=target_transform)
        self.train = train  # training set or test set

        # ------- We can ingore this block -------
        if download:
            self.download()

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' +' You can use download=True to download it')
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file
        # -----------------------------------------------

        # We change the lines below; these specify how the data are loaded.
        # self.data contain images (type 'torch.Tensor' of [num_images, H, W]) 
        # self.targets contain labels (class ids) (type 'torch.Tensor' of [num_images]) 
        # images and labels are stored in the same 
        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))

        # We use only the images and lables whose indeces are in the range of 0..limit_data-1.
        if not limit_data is None:
          self.data    = self.data[ :limit_data, :,:]
          self.targets = self.targets[:limit_data]
          if self.train:
            print("[WRN]: Trainig Data is limited, only the first "+str(self.data.size(0))+" samples will be used.")
          else:
            print("[WRN]: Test Data is limited, only the first "   +str(self.data.size(0))+" samples will be used.")            


    def __getitem__(self, index):
        # We extract the image and label of the specified 'index'.
        img, target = self.data[index], int(self.targets[index])

        # Prepare for self.transform below.
        img = Image.fromarray(img.numpy(), mode='L')

        # Transform img.
        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.data)

    urls = [
        'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
        'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
    ]
    training_file = 'training.pt'
    test_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    @property
    def train_labels(self):
        warnings.warn("train_labels has been renamed targets")
        return self.targets

    @property
    def test_labels(self):
        warnings.warn("test_labels has been renamed targets")
        return self.targets

    @property
    def train_data(self):
        warnings.warn("train_data has been renamed data")
        return self.data

    @property
    def test_data(self):
        warnings.warn("test_data has been renamed data")
        return self.data

    @property
    def raw_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, self.__class__.__name__, 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder,
                                            self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder,
                                            self.test_file)))
        
    def download(self):
        """Download the MNIST data if it doesn't exist in processed_folder already."""

        if self._check_exists():
            return

        makedir_exist_ok(self.raw_folder)
        makedir_exist_ok(self.processed_folder)

        # download files
        for url in self.urls:
            filename = url.rpartition('/')[2]
            download_and_extract_archive(url, download_root=self.raw_folder, filename=filename)

        # process and save as torch files
        print('Processing...')

        training_set = (
            read_image_file(os.path.join(self.raw_folder, 'train-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 'train-labels-idx1-ubyte'))
        )
        test_set = (
            read_image_file(os.path.join(self.raw_folder, 't10k-images-idx3-ubyte')),
            read_label_file(os.path.join(self.raw_folder, 't10k-labels-idx1-ubyte'))
        )
        with open(os.path.join(self.processed_folder, self.training_file), 'wb') as f:
            torch.save(training_set, f)
        with open(os.path.join(self.processed_folder, self.test_file), 'wb') as f:
            torch.save(test_set, f)

        print('Done!')

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")

def read_label_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 1)
    return x.long()


def read_image_file(path):
    with open(path, 'rb') as f:
        x = read_sn3_pascalvincent_tensor(f, strict=False)
    assert(x.dtype == torch.uint8)
    assert(x.ndimension() == 3)
    return x

def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def open_maybe_compressed_file(path):
    """Return a file object that possibly decompresses 'path' on the fly.
       Decompression occurs when argument `path` is a string and ends with '.gz' or '.xz'.
    """
    if not isinstance(path, torch._six.string_classes):
        return path
    if path.endswith('.gz'):
        import gzip
        return gzip.open(path, 'rb')
    if path.endswith('.xz'):
        import lzma
        return lzma.open(path, 'rb')
    return open(path, 'rb')


def read_sn3_pascalvincent_tensor(path, strict=True):
    """Read a SN3 file in "Pascal Vincent" format (Lush file 'libidx/idx-io.lsh').
       Argument may be a filename, compressed filename, or file object.
    """
    # typemap
    if not hasattr(read_sn3_pascalvincent_tensor, 'typemap'):
        read_sn3_pascalvincent_tensor.typemap = {
            8: (torch.uint8, np.uint8, np.uint8),
            9: (torch.int8, np.int8, np.int8),
            11: (torch.int16, np.dtype('>i2'), 'i2'),
            12: (torch.int32, np.dtype('>i4'), 'i4'),
            13: (torch.float32, np.dtype('>f4'), 'f4'),
            14: (torch.float64, np.dtype('>f8'), 'f8')}
    # read
    with open_maybe_compressed_file(path) as f:
        data = f.read()
    # parse
    magic = get_int(data[0:4])
    nd = magic % 256
    ty = magic // 256
    assert nd >= 1 and nd <= 3
    assert ty >= 8 and ty <= 14
    m = read_sn3_pascalvincent_tensor.typemap[ty]
    s = [get_int(data[4 * (i + 1): 4 * (i + 2)]) for i in range(nd)]
    parsed = np.frombuffer(data, dtype=m[1], offset=(4 * (nd + 1)))
    assert parsed.shape[0] == np.prod(s) or not strict
    return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [28]:

# simple network

import torch
import torch.nn as nn
import torch.nn.functional as F

# Build a simple network
class simple_network(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(simple_network, self).__init__()
    self.relu = nn.ReLU()
    self.dropout1 = nn.Dropout(0.2)
    # Fully connected layers
    self.fc1 = nn.Linear(784, 512)
    nn.init.kaiming_uniform_(self.fc1.weight)
    self.fc2 = nn.Linear(512, num_class)
    nn.init.kaiming_uniform_(self.fc2.weight)
 
    
  def forward(self, x):
    x = self.relu(self.fc1(x))                   
    x = self.relu(self.fc2(x))     
    x = self.dropout1(x)

    return x
  

class simple_network1(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(simple_network1, self).__init__()
    self.relu = nn.ReLU()
    self.dropout1 = nn.Dropout(0.2)
    self.fc1 = nn.Linear(784, 512)
    nn.init.kaiming_uniform_(self.fc1.weight)
    self.fc2 = nn.Linear(512, 128)
    self.fc3 = nn.Linear(128, num_class)
    nn.init.kaiming_uniform_(self.fc2.weight)
    
    
  def forward(self, x):
    x = self.relu(self.fc1(x))                 
    x = self.relu(self.fc2(x))  
    x = self.relu(self.fc3(x))  
    x = self.dropout1(x)
    
    return x
      
     

In [29]:
# LeNet

import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    
  def __init__(self, input_dim=1, num_class=10):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(input_dim, 20,  kernel_size=5, stride=1, padding=0)
    self.conv2 = nn.Conv2d(    20,    50,  kernel_size=5, stride=1, padding=0)
    self.fc1 = nn.Linear(800, 500)
    self.fc2 = nn.Linear(500, num_class)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.conv1(x))                         # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)         # Pooling with 2x2 window
    x = self.relu(self.conv2(x))                         # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)         # Pooling with 2x2 window
    b,c,h,w = x.size()                                   # batch, channels, height, width
    x = x.view(b, -1)                                    # flatten the tensor x
    x = self.relu(self.fc1(x))                           # fc-> ReLU
    x = self.fc2(x)                                      # fc
    
    return x



In [30]:
# CNN1

import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN1(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(CNN1, self).__init__()

    self.conv1 = nn.Conv2d(input_dim, 20,  kernel_size=5, stride=1, padding=0)  
    self.conv1_drop = nn.Dropout2d()
    self.conv2 = nn.Conv2d(    20,    50,  kernel_size=5, stride=1, padding=0)   
    self.conv2_drop = nn.Dropout2d()
    self.conv3 = nn.Conv2d(    50,    100,  kernel_size=2, stride=1, padding=0) 
    self.conv3_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(800, 500)
    self.fc2 = nn.Linear(500, num_class)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.conv1(x))                                   # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)                  # Pooling with 2x2 window
    x = self.relu(self.conv2(x))                                  # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)                  # Pooling with 2x2 window
    b,c,h,w = x.size()                                            # batch, channels, height, width
    x = x.view(b, -1)                                             # flatten the tensor x
    x = self.relu(self.fc1(x))                                    # fc-> ReLU
    x = self.fc2(x)                                               # fc
    return x


In [31]:
#CNN2 

import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN2(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(CNN2, self).__init__()


    self.conv1 = nn.Conv2d(input_dim, 20,  kernel_size=5, stride=1, padding=0)  
    self.conv1_drop = nn.Dropout2d()
    self.conv2 = nn.Conv2d(    20,    50,  kernel_size=5, stride=1, padding=0)   
    self.conv2_drop = nn.Dropout2d()
    self.conv3 = nn.Conv2d(    50,    100,  kernel_size=2, stride=1, padding=0) 
    self.conv3_drop = nn.Dropout2d()
    self.conv4 = nn.Conv2d(    50,    100,  kernel_size=2, stride=1, padding=0) 
    self.fc1 = nn.Linear(800, 500)
    self.fc2 = nn.Linear(500, num_class)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.conv1(x))                                   # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)                  # Pooling with 2x2 window
    x = self.relu(self.conv2(x))                                  # Conv.-> ReLU
    x = F.max_pool2d(x, kernel_size=2, stride=2)                  # Pooling with 2x2 window

    b,c,h,w = x.size()                                            # batch, channels, height, width
    x = x.view(b, -1)                                             # flatten the tensor x
    x = self.relu(self.fc1(x))                                    # fc-> ReLU
    x = self.fc2(x)                                               # fc
   
    return x


In [32]:
#CNN3

import torch
import torch.nn as nn
import torch.nn.functional as F

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class CNN3(nn.Module):
  def __init__(self, input_dim=1, num_class=10):
    super(CNN3, self).__init__()

    self.conv1 = nn.Conv2d(input_dim, 64,kernel_size=7,stride=2, padding=0) 
    self.conv2 = BasicBlock(inplanes=64,  planes=64)


    self.fc1 = nn.Linear(760384,  500)
    self.fc2 = nn.Linear(500, num_class)
    

  def forward(self, x):

    x = self.conv1(x)                               # Conv.-> ReLU
    x = self.conv2(x)                               # Conv.-> ReLU
    b,c,h,w = x.size()                                             # batch, channels, height, width
    #print(x.size())
    x = x.view(b, -1)                                             # flatten the tensor x
    x = self.fc1(x)                                   #fc-> ReLU
    x = self.fc2(x)                                               # fc
    return x


In [33]:
import torchvision.models as  models
import torch.nn as nn
from  torchsummary import  summary

# Resnet
'''
net = models.resnet(pretrained=False, progress=True) 
net = models.resnet34(pretrained=False, progress=True)
net = models.resnet34(pretrained=False, progress=True)
net = models.resnet50(pretrained=False, progress=True)
net = models.resnet101(pretrained=False, progress=True
net = models.resnet152(pretrained=False, progress=True)
 

net = models.resnet18(pretrained=False, progress=True)                       
net.fc = nn.Linear(  in_features=512,  out_features=10  )
net.conv1=nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
net=net.cuda()
#rint(net)


summary(net,input_size =( 1,224,224)) 

                       

# Alexnet

net = models.alexnet( pretrained=False, progress=True)
net.features[0] = nn.Conv2d(1,64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
net.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
net.cuda()
print(net)

# VGG

net = models.vgg11(pretrained=False, progress=True ) 
net.features[0] = nn.Conv2d(1,64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
net.classifier[6] =nn.Linear(in_features=4096, out_features=10, bias=True)
net.cuda()
print(net)

'''

# mobilenetv2

net = models.MobileNetV2()
net.features[0][0]= nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
net.classifier[1]=nn.Linear(in_features=1280, out_features=10, bias=True)   
net.cuda()
summary(net,input_size =( 1,224,224)) 

#print(net)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             288
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [34]:
# Prepare the trainloader/testloader for training/validating network 

from   torchvision import datasets as datasets
import torchvision.transforms as transforms
import torch.utils as utils
import matplotlib.pyplot as plt
import torch
import torchvision

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize((0.1307,), 
                                                                                                (0.3081,))])                                                                                                                                                                                  
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

#mnist_train = MyMNIST('./data', train=True,  download=True, transform=transform)
mnist_train = MyMNIST('./data', train=True,  download=True, transform=transform, limit_data=1000)
mnist_test  = MyMNIST('./data', train=False, download=True, transform=transform)
trainloader = utils.data.DataLoader(mnist_train, batch_size=20, shuffle=True,  num_workers=4)
testloader  = utils.data.DataLoader(mnist_test, batch_size=20, shuffle=False, num_workers=4)

[WRN]: Trainig Data is limited, only the first 1000 samples will be used.


In [35]:
# Initialize the network
#net = simple_network().cuda()
#net =  simple_network1().cuda()
#net = LeNet().cuda()
#net = CNN1().cuda()
#net = CNN2().cuda()
#net = CNN3().cuda()

#print(net)
#summary(net,input_size =( 1,224,224)) 

In [36]:
# Prepare the optimizer and loss func. 

import torch.optim as optim
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(net.parameters())
epoch = 0

In [37]:
#evaluate the network 

def evaluate_model():
  print("Testing the network...")
  net.eval()
  total_num   = 0
  correct_num = 0
  for test_iter, test_data in enumerate(testloader):
    # Get one batch of test samples
    inputs, labels = test_data    
    bch = inputs.size(0)
    #nput = inputs.view(bch, 1,28,28)
    #nputs = inputs.view(bch, -1)
    # Move inputs and labels into GPU
    inputs = inputs.cuda()
    labels = torch.LongTensor(list(labels)).cuda()
    # Forward
    # int(inputs)
    outputs = net(inputs)   
    # Get predicted classes
    _, pred_cls = torch.max(outputs, 1)
    correct_num+= (pred_cls == labels).float().sum().item()
    total_num+= bch
  net.train() 
  print("Accuracy: "+"%.5f"%(correct_num/float(total_num)))

In [38]:
epoch_size = 20

for epoch_idx in range(epoch_size):
  running_loss = 0.0
  ct_num       = 0
  for iteration, data in enumerate(trainloader):
    # Take the inputs and the labels for 1 batch.
    inputs, labels = data
    #print(length(inputs))
    bch = inputs.size(0)
    #inputs = inputs.view(bch, 1,28,28)
    #inputs = inputs.view(bch,-1)
    # Move inputs and labels into GPU
    inputs = inputs.cuda()
    labels = labels.cuda()
    #print("label"  )
    #print(labels.size())
    # Remove old gradients for the optimizer.
    optimizer.zero_grad()
    # Compute result (Forward)
    #print(inputs.size())
    outputs = net(inputs)
    # Compute loss
    loss    = loss_func(outputs, labels)
    # Calculate gradients (Backward)
    loss.backward()
    # Update parameters
    optimizer.step()
    #with torch.no_grad():
    running_loss += loss.item()
    ct_num+= 1
    if iteration%20 == 19:
      print("[Epoch: "+str(epoch+1)+"]"" --- Iteration: "+str(iteration+1)+", Loss: "+str(running_loss/ct_num)+'.')

    # Test
    if epoch%3 == 2 and iteration%100 == 99:
      evaluate_model()
    
  epoch += 1

[Epoch: 1] --- Iteration: 20, Loss: 2.3308866381645204.
[Epoch: 1] --- Iteration: 40, Loss: 2.2998652875423433.
[Epoch: 2] --- Iteration: 20, Loss: 2.16077715754509.
[Epoch: 2] --- Iteration: 40, Loss: 2.1051378190517425.
[Epoch: 3] --- Iteration: 20, Loss: 1.9463570773601533.
[Epoch: 3] --- Iteration: 40, Loss: 1.8766456872224808.
[Epoch: 4] --- Iteration: 20, Loss: 1.6930298805236816.
[Epoch: 4] --- Iteration: 40, Loss: 1.6324333816766738.
[Epoch: 5] --- Iteration: 20, Loss: 1.3538878202438354.
[Epoch: 5] --- Iteration: 40, Loss: 1.3116397827863693.
[Epoch: 6] --- Iteration: 20, Loss: 1.1282144069671631.
[Epoch: 6] --- Iteration: 40, Loss: 1.0493448674678802.
[Epoch: 7] --- Iteration: 20, Loss: 0.8580922484397888.
[Epoch: 7] --- Iteration: 40, Loss: 0.8003025472164154.
[Epoch: 8] --- Iteration: 20, Loss: 0.592923353612423.
[Epoch: 8] --- Iteration: 40, Loss: 0.5681075170636177.
[Epoch: 9] --- Iteration: 20, Loss: 0.39638520777225494.
[Epoch: 9] --- Iteration: 40, Loss: 0.383499386534

In [39]:
evaluate_model()

Testing the network...
Accuracy: 0.91580


In [40]:
'''

import torchvision.models as  models
net = models.Fire()
net = models.BasicBlock()
print(net)

'''


'\n\nimport torchvision.models as  models\nnet = models.Fire()\nnet = models.BasicBlock()\nprint(net)\n\n'